# Configurações da biblioteca pandas_datareader

In [ ]:
! pip install --upgrade numpy
! pip install --upgrade pandas
! pip install --upgrade pandas-datareader

# Obtenção dos dados

In [146]:
import pandas_datareader as dados
import pandas as pd

#ativo = "AMZN"  # https://finance.yahoo.com/ ##Amazon
#ativo = "NTCO3.SA"  # https://finance.yahoo.com/ ##Natura
#ativo = "VALE3.SA"  # https://finance.yahoo.com/ ##Vale

ativo = "AMZN"  # https://finance.yahoo.com/

##Dados_Ativo = os dados biaxados da api iniciando e finalizando o dataset

dados_do_ativo = dados.DataReader(ativo, start="2015-01-01",end="2021-08-24", data_source='yahoo')
#dados_do_ativo = dados.DataReader(ativo, start="2018-01-01",end="2022-11-30", data_source='yahoo')
#dados_do_ativo = dados.DataReader(ativo, start="2018-01-01",end="2022-11-30", data_source='yahoo')

#dados_do_ativo = dados.DataReader(ativo, start="2022-11-01",end="2022-11-20", data_source='yahoo')

In [ ]:
##### Verificando os valores e colunas #####

#print(dados_do_ativo.isna().any()) # Exibe como True ou False os valores nan do Dataframe
#print(dados_do_ativo.isna().sum()) # Exibe a soma dos valores nan no Dataframe
#dados_do_ativo.head()
#dados_do_ativo.info(verbose=True)
#print(dados_do_ativo.info())
dados_do_ativo.shape
#dados_do_ativo.isna().sum()
#dados_do_ativo.isnull().sum()

In [ ]:
dados_do_ativo

# Visualização dos dados

In [ ]:
import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
#plt.style.use('default')

plt.figure(figsize=(16,8))
plt.title('Histórico de fechamento ' + ativo)
plt.plot(dados_do_ativo['Close'])
plt.xlabel('Data')
plt.ylabel('Preço de fechamento')
plt.show()

# Preparação dos dados

In [149]:
##Cotacoes_df = a coluna Close de dados do ativo
cotacoes_df = dados_do_ativo.filter(['Close'])
cotacoes = cotacoes_df.values

from sklearn.preprocessing import MinMaxScaler

##Normaliza os dados/valores entre 0 e 1
normalizador = MinMaxScaler(feature_range=(0, 1)) 
cotacoes_normalizadas = normalizador.fit_transform(cotacoes)

In [ ]:
#cotacoes_normalizadas
cotacoes_normalizadas

# Separação dos dados para treinamento

In [150]:
##Separação dos dados de Traino 

from math import ceil ## Funcao para arredondar os valores

dias_treinamento = ceil( len(cotacoes) *.8) #Mutiplica por 8, pra retirar 80% dos dados
cotacoes_treinamento = cotacoes_normalizadas[0:dias_treinamento]##Cotações treino = os 80% primeiros dados do df cotacoes normalizados

x_treino=[]
y_treino = []
##percorre o periodo de tempo, separando os valores em x,y
for i in range(30,len(cotacoes_treinamento)):
    x_treino.append(cotacoes_treinamento[i-30:i])
    y_treino.append(cotacoes_treinamento[i])

import numpy as np

x_treino, y_treino = np.array(x_treino), np.array(y_treino) ##Configura os dados para o array

x_treino = np.reshape(x_treino, (len(x_treino),30,1)) ## configura os dias e o betside

# Criação do modelo

In [151]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
##Camada Sequencial
model.add(LSTM(units=50, return_sequences=True,input_shape=(30,1)))
model.add(LSTM(units=50, return_sequences=False))
#Camada Densa
model.add(Dense(units=25))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')



# Treinamento do modelo

In [ ]:
model.fit(x_treino, y_treino, batch_size=1, epochs=3)

# Separação e preparação dos dados para o teste

In [153]:
##Separação dos dados de Teste

cotacoes_teste = cotacoes_normalizadas[dias_treinamento - 30:]

x_teste = []
y_teste =  cotacoes[dias_treinamento:] 
for i in range(30,len(cotacoes_teste)):
    x_teste.append(cotacoes_teste[i-30:i])


x_teste = np.array(x_teste)
x_teste = np.reshape(x_teste, (len(x_teste),30,1))

# Teste (predição)

In [ ]:
predictions = model.predict(x_teste) ##recebe a predicao de todos os valores de x-teste

predictions = normalizador.inverse_transform(predictions)##Normaliza os valores e faz a inversao dos valores que estavam entre 0 e 1 para valores normais

In [23]:
#predictions

# Cálculo do erro (diferença entre predição e valor real)

In [ ]:
rmse=np.sqrt(np.mean(((predictions- y_teste)**2)))
rmse

# Gráfico de comparação

In [ ]:
treino = cotacoes_df[:dias_treinamento]
valido = cotacoes_df[dias_treinamento:]
valido['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('Ativo ' + ativo)
plt.xlabel('Data')
plt.ylabel('Preço de fechamento (R$)')
plt.plot(treino['Close'], color = 'black')
plt.plot(valido['Close'], color = 'green')
plt.plot(valido['Predictions'], color = 'red')
plt.legend(['Histórico', 'Valor real', 'Predição'])
plt.show()

# Teste atual

In [ ]:
dados_do_ativo = dados.DataReader(ativo, start="2021-08-23",end="2021-10-05", data_source='yahoo') ## Nova base de dados inicia em 23-08-2021 finaliza em 05-10-2021

x = dados_do_ativo.filter(['Close']) ## pega a loluna de fechamento
x = normalizador.fit_transform(x) #normaliza os dados
x = np.array(x)[:-1] # remove o ultimo dia da base, nesse caso remove o dia 05/10
x = np.reshape(x, (1,30,1)) ##Configuração de dados

prediction = model.predict(x) #dias de selecionados para o teste

prediction = normalizador.inverse_transform(prediction)#faz a inversao dos valores
real = dados_do_ativo.filter(['Close']).values[-1] #pega o valor real do utimo dia da base de dados(valor que foi retirado)

print('predição: ',prediction)
print('real: ',real)

In [28]:
#dados_do_ativo